In [1]:
# Import Required Libraries
import numpy as np
from autograd import grad, hessian

In [1]:
# Bisection Method 
def bisection_method_for_min(f, a, b, tol=1e-5, max_iter=100):
    """
    Perform the Bisection Method to estimate the local minimum of a function.
    This assumes the local minimum lies in the interval [a, b].
    
    Parameters:
    f (function): The function for which we are trying to find the local minimum.
    a (float): The start of the interval [a, b].
    b (float): The end of the interval [a, b].
    tol (float): The tolerance level for the minimum (default is 1e-5).
    max_iter (int): The maximum number of iterations (default is 100).

    Returns:
    float: The approximate location of the local minimum within the given tolerance.
    """
    iter_count = 0
    while (b - a) / 2.0 > tol and iter_count < max_iter:
        # Find the midpoint
        midpoint = (a + b) / 2.0
        
        # Derivative of f(alpha) approximated by central difference method
        f_prime = (f(midpoint + tol) - f(midpoint - tol)) / (2 * tol)
        
        # Print the current step
        print(f"Iteration {iter_count}: a = {a}, b = {b}, midpoint = {midpoint}, f'(midpoint) = {f_prime}")
        
        # Check if the derivative is close to 0 (indicating a minimum)
        if abs(f_prime) < tol:
            return midpoint
        
        # Narrow down the interval based on the slope
        if f_prime > 0:
            b = midpoint  # Minimum is to the left
        else:
            a = midpoint  # Minimum is to the right
        
        iter_count += 1

    return (a + b) / 2.0

In [3]:
# Exact Line Search using Bisection Method
def line_search_bisection(f, a, b, x0, direction, tol=1e-5, max_iter=100):
    """
    Perform a line search using the bisection method to find the optimal step size (alpha)
    for minimizing a function f along a given descent direction.

    This function uses the bisection method to find the value of alpha that minimizes the 
    function f(x0 + alpha * direction). It does this by iteratively refining the interval [a, b]
    where the derivative of the function with respect to alpha is close to zero.

    Parameters:
    f (function): A function that takes (alpha, x0, direction) and returns f(x0 + alpha * direction).
    a (float): The lower bound of the interval for alpha.
    b (float): The upper bound of the interval for alpha.
    x0 (numpy array): The current point in the optimization process (starting point).
    direction (numpy array): The descent direction along which the function is minimized.
    tol (float): The tolerance level for the derivative of f(alpha). Default is 1e-5.
    max_iter (int): The maximum number of iterations allowed for the bisection search. Default is 100.

    Returns:
    float: The optimal alpha that minimizes f(x0 + alpha * direction) within the given tolerance.
    """
    iter_count = 0
    while (b - a) / 2.0 > tol and iter_count < max_iter:
        midpoint = (a + b) / 2.0
        
        # Derivative of f(alpha) approximated by central difference method
        f_prime = (f(midpoint + tol, x0, direction) - f(midpoint - tol, x0, direction)) / (2 * tol)
        
        print(f"Iteration {iter_count}: alpha = {midpoint}, f'(alpha) = {f_prime}")
        
        if abs(f_prime) < tol:  # Stop if the derivative is close to zero
            return midpoint
        
        if f_prime > 0:
            b = midpoint  # Narrow down to the left
        else:
            a = midpoint  # Narrow down to the right
        
        iter_count += 1

    return (a + b) / 2.0


In [ ]:
# Backtrackign Approximate Line Search
def backtracking_line_search(f, grad_f, x_i, d, alpha=1, p=0.5, c=1e-4):
    """
    Backtracking line search algorithm to find the step size that satisfies the Wolfe conditions.

    Parameters:
    f (function): The objective function.
    grad_f (function): The gradient of the objective function.
    x_i (numpy array): The current point.
    d (numpy array): The descent direction.
    alpha (float): The initial step size (default is 1).
    p (float): The factor for reducing the step size (default is 0.5).
    c (float): The constant for the sufficient decrease condition (default is 1e-4).

    Returns:
    float: The optimal step size alpha.
    """
    # While the sufficient decrease condition is violated
    while f(x_i + alpha * d) > f(x_i) + c * alpha * grad_f(x_i).T.dot(d):
        # Reduce the step size
        alpha *= p
    
    return alpha


In [2]:
# Gradient Descent Algortithm with Gradient Magnitude Termination Condition
def gradient_descent(f, x0, epsilon=1e-6, max_iter=1000):
    """
    Gradient descent algorithm with line search.
    
    Parameters:
    f (function): Objective function to minimize.
    grad_f (function): Gradient of the objective function.
    x0 (numpy array): Initial guess for the variable.
    epsilon (float): Tolerance for stopping.
    max_iter (int): Maximum number of iterations.

    Returns:
    x (numpy array): Final variable vector.
    f(x) (float): Function value at the final point.
    """
    x_i = x0
    grad_f = grad(f)
    for i in range(max_iter):
        # Compute the descent direction (negative gradient normalized)
        grad = grad_f(x_i)
        d_i = -grad / np.linalg.norm(grad)
        
        # Line search to find step size (using backtracking line search)
        alpha_i = backtracking_line_search(f, grad_f, x_i, d_i)
        
        # Update the variable
        x_i = x_i + alpha_i * d_i
        
        # Check termination condition: gradient magnitude
        if np.linalg.norm(grad) < epsilon:
            print(f"Converged after {i+1} iterations.")
            break
    
    return x_i, f(x_i)

In [ ]:
# Gradient Descent with Noise
def gradient_descent_with_noise(f, x0, epsilon=1e-6, max_iter=1000):
    """
    Gradient descent algorithm with Gaussian noise and fixed step size.
    
    Parameters:
    f (function): Objective function to minimize.
    grad_f (function): Gradient of the objective function.
    x0 (numpy array): Initial guess for the variable.
    epsilon (float): Tolerance for stopping.
    max_iter (int): Maximum number of iterations.

    Returns:
    x (numpy array): Final variable vector.
    f(x) (float): Function value at the final point.
    """
    x_i = x0
    grad_f = grad(f)
    for i in range(1, max_iter + 1):
        # Compute the descent direction (negative gradient normalized)
        grad = grad_f(x_i)
        d_i = -grad / np.linalg.norm(grad)
        
        # Set fixed step size
        alpha_i = 1 / i
        
        # Add Gaussian noise with decreasing standard deviation
        noise = np.random.normal(0, 1 / (i**3), size=x_i.shape)
        
        # Update the variable with noisy descent
        x_i = x_i + alpha_i * d_i + noise
        
        # Check termination condition: gradient magnitude
        if np.linalg.norm(grad) < epsilon:
            print(f"Converged after {i} iterations.")
            break
    
    return x_i, f(x_i)

In [ ]:
# Gradient Descent  with Momentum
def gradient_descent_with_momentum(f, x0, alpha=0.1, beta=0.9, epsilon=1e-6, max_iter=1000):
    """
    Gradient descent algorithm with momentum.
    
    Parameters:
    f (function): Objective function to minimize.
    grad_f (function): Gradient of the objective function.
    x0 (numpy array): Initial guess for the variable.
    alpha (float): Step size (default is 0.1).
    beta (float): Momentum coefficient (default is 0.9).
    epsilon (float): Tolerance for stopping.
    max_iter (int): Maximum number of iterations.

    Returns:
    x (numpy array): Final variable vector.
    f(x) (float): Function value at the final point.
    """
    x_i = x0
    grad_f = grad(f)
    v = np.zeros_like(x0)  # Initialize the velocity term
    
    for i in range(1, max_iter + 1):
        # Compute the descent direction (negative gradient)
        grad = grad_f(x_i)
        
        # Update velocity
        v = beta * v - alpha * grad
        
        # Update the variable
        x_i = x_i + v
        
        # Check termination condition: gradient magnitude
        if np.linalg.norm(grad) < epsilon:
            print(f"Converged after {i} iterations.")
            break
    
    return x_i, f(x_i)

In [ ]:
def newton_method(f, x0, epsilon=1e-6, max_iter=1000):
    """
    Perform Newton's method to find the minimum of a function.

    Parameters:
    -----------
    f : callable
        The objective function to minimize. It must take a vector as input and return a scalar.
    x0 : ndarray
        The initial guess for the optimization, a NumPy array of floats.
    epsilon : float, optional (default=1e-6)
        The convergence tolerance. Iteration stops when the norm of the gradient is less than epsilon.
    max_iter : int, optional (default=1000)
        The maximum number of iterations allowed.

    Returns:
    --------
    x_i : ndarray
        The final estimate for the minimum point of the function.
    steps : list of ndarray
        A list containing the points visited during the optimization process, representing the path of the optimization.
    """
    # Ensure x0 is a float-type NumPy array
    x_i = x0
    steps = [x_i]
    
    # Compute gradient and Hessian functions
    grad_f = grad(f)
    hessian_f = hessian(f)
    
    for i in range(max_iter):
        # Compute gradient and Hessian at the current point
        grad_i = grad_f(x_i)
        hessian_i = hessian_f(x_i)
        
        # Ensure Hessian is invertible
        try:
            step = np.linalg.inv(hessian_i) @ grad_i
        except np.linalg.LinAlgError:
            print("Hessian is singular, stopping optimization.")
            break
        
        # Update step
        x_i = x_i - step
        steps.append(x_i)
        # Check for convergence
        if np.linalg.norm(grad_i) <= epsilon:
            print(f"Converged after {i + 1} steps")
            break
    else:
        print("Max iterations reached without convergence.")
    print(f"Steps: {steps}")
    print(f"Final Point: {x_i}, f(x) = {f(x_i)}")
    return x_i,steps